# PAI

> Pipeline to compute PAI


In [ ]:
#| default_exp pai

In [ ]:
#| hide
from nbdev.showdoc import *

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/mtinti/miniforge3/envs/stella_seq/lib/python3.9/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mtinti/miniforge3/envs/stella_seq/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/Users/mtinti/miniforge3/envs/stella_seq/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 301, in dispatch_control
    async with self._control_lock:
  File "/Users/mtinti/miniforge3/envs/stella_seq/lib/python3.9/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
  File "/Users/mtinti/miniforge3/envs/stella_seq/lib/python3.9/site-packages/traitlets/traitlets.p

In [ ]:
#| export
import matplotlib.pyplot as plt
from typing import Dict, Union
from pyteomics import parser
from tqdm import tqdm
from pyteomics import mass
from pathlib import Path
from Bio import SeqIO
import pandas as pd
import polars as pl
import numpy as np

from protein_cutter.core import digest_to_empai_set
from protein_cutter.core import collapse_empai_entries
from protein_cutter.core import flag_proprietary_peptides_from_set
from protein_cutter.core import flag_proprietary_from_pg
from protein_cutter.core import load_peptides_from_fasta
from protein_cutter.core import load_fasta

Repo root: /Users/mtinti/git_projects/protein_cutter
Test data dir: /Users/mtinti/git_projects/protein_cutter/test_data
Test data exists: True
/Users/mtinti/git_projects/protein_cutter/nbs


In [ ]:
#| export
from pathlib import Path
import os
import yaml
# Get the repository root
if 'GITHUB_WORKSPACE' in os.environ:
    # In GitHub Actions
    REPO_ROOT = Path(os.environ['GITHUB_WORKSPACE'])
else:
    # Local development - find repo root
    REPO_ROOT = Path.cwd()
    while not (REPO_ROOT / 'settings.ini').exists():
        if REPO_ROOT == REPO_ROOT.parent:
            REPO_ROOT = Path.cwd()  # Fallback
            break
        REPO_ROOT = REPO_ROOT.parent

TEST_DATA = REPO_ROOT / 'test_data'
CONFIG_FILES = REPO_ROOT / 'config_files'
print(f"Repo root: {REPO_ROOT}")
print(f"Test data dir: {TEST_DATA}")
print(f"Test data exists: {TEST_DATA.exists()}")

Repo root: /Users/mtinti/git_projects/protein_cutter
Test data dir: /Users/mtinti/git_projects/protein_cutter/test_data
Test data exists: True


In [ ]:
#| export
with open(CONFIG_FILES / 'config.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
print(config)   

{'min_pep_length': 6, 'max_pep_length': 52, 'min_mz_range': 200, 'max_mz_range': 4000, 'enzyme': 'trypsin_full', 'input_pep': 'test_spectronaut_pep_out.tsv', 'input_prot': 'test_spectronaut_prot_out.tsv', 'input_fasta': 'test_sequence.fa'}


In [ ]:
#| export
infile = TEST_DATA/'pipeline_test/prot.fa'
#infile = '../datasets/spectronaut_protein.fasta'
fasta_dict = load_fasta(infile)

/Users/mtinti/git_projects/protein_cutter/nbs


In [ ]:
#| export
def get_observable(
    sequence: str,
    enzyme: str = 'trypsin_full') -> set[str]:
    """
    Get the set of observable peptides from in silico digestion.
    
    Performs in silico digestion of a protein sequence and returns
    the set of theoretically observable peptides. This is used as
    the denominator (N_observable) in PAI and emPAI calculations.
    
    Parameters
    ----------
    sequence : str
        Protein amino acid sequence to digest.
    enzyme : str
        Enzyme name for in silico digestion. Must be a key in
        protease_dict. Default is trypsin_full.
    
    Returns
    -------
    set of str
        Set of unique observable peptide sequences after digestion
        and filtering by m/z range.
    
    Notes
    -----
    The function applies the following constraints:
    
    - Zero missed cleavages (fully cleaved peptides only)
    - m/z range filter of 200 to 4000 Da
    - Collapsed entries to remove charge state duplicates
    
    """
    observable_peptides = collapse_empai_entries(
        digest_to_empai_set(
            sequence=sequence,
            enzyme=enzyme,
            missed_cleavages=0,
            mz_range=(200, 4000),
        )
    )
    return observable_peptides

In [ ]:
#| export
#infile = "../datasets/spectronaut_peptide_report.tsv"
infile = TEST_DATA/'pipeline_test/pep_report.tsv'
df_pep  = pl.read_csv(infile,separator='\t').to_pandas()
df_pep = df_pep[df_pep['PEP.NrOfMissedCleavages']==0]
df_pep.head()

,,R.Condition,R.FileName,R.Replicate,PG.ProteinAccessions,PG.ProteinDescriptions,PG.ProteinNames,PG.Coverage,PG.IsSingleHit,PG.Qvalue,PG.Quantity,PEP.IsProteotypic,PEP.NrOfMissedCleavages,PEP.StrippedSequence,EG.PrecursorId,EG.Identified,EG.Qvalue,EG.ApexRT,EG.TotalQuantity (Settings),FG.Charge
0,0,Cell-34C,331_2022_GBR-Cell-34C-A,1,Tb427_000008800.1-p1,transcript=Tb427_000008800.1 | gene=Tb427_0000...,None,16.8%,False,0.0,551307.5625,False,0,MTTSECYAPQLDLLMNSDHK,_MTTSEC[Carbamidomethyl (C)]YAPQLDLLMNSDHK_.3,True,4.721150e-27,86.696732,1.350588e+06,3
1,1,Cell-34C,331_2022_GBR-Cell-34C-A,1,Tb427_000008800.1-p1,transcript=Tb427_000008800.1 | gene=Tb427_0000...,None,16.8%,False,0.0,551307.5625,False,0,MTTSECYAPQLDLLMNSDHK,_M[Oxidation (M)]TTSEC[Carbamidomethyl (C)]YAP...,True,1.694105e-28,86.696732,1.334962e+06,3
2,2,Cell-34C,331_2022_GBR-Cell-34C-A,1,Tb427_000008800.1-p1,transcript=Tb427_000008800.1 | gene=Tb427_0000...,None,16.8%,False,0.0,551307.5625,False,0,MTTSECYAPQLDLLMNSDHK,_MTTSEC[Carbamidomethyl (C)]YAPQLDLLM[Oxidatio...,True,5.703590e-22,86.686089,3.219499e+05,3
3,3,Cell-34C,331_2022_GBR-Cell-34C-A,1,Tb427_000008800.1-p1,transcript=Tb427_000008800.1 | gene=Tb427_0000...,None,16.8%,False,0.0,551307.5625,False,0,MTTSECYAPQLDLLMNSDHK,_MTTSEC[Carbamidomethyl (C)]YAPQLDLLMN[Deamida...,True,4.645979e-31,89.731918,8.812225e+04,3
4,4,Cell-34C,331_2022_GBR-Cell-34C-A,1,Tb427_000008800.1-p1,transcript=Tb427_000008800.1 | gene=Tb427_0000...,None,16.8%,False,0.0,551307.5625,False,0,MTTSECYAPQLDLLMNSDHK,_MTTSEC[Carbamidomethyl (C)]YAPQLDLLMNSDHK_.2,True,4.328258e-07,86.766289,5.902369e+04,2


In [ ]:
#| export
#infile = "../datasets/spectronaut_protein_report.tsv"
infile = TEST_DATA/'pipeline_test/prot_report.tsv'
df_prot = pl.read_csv(infile, separator='\t').to_pandas()
df_prot.head()

,,PG.ProteinGroups,[1] 331_2022_GBR-Cell-34C-A.raw.PG.Quantity,[2] 331_2022_GBR-Cell-34C-B.raw.PG.Quantity,[3] 331_2022_GBR-Cell-34C-C.raw.PG.Quantity,[4] 331_2022_GBR-Cell-37C-A.raw.PG.Quantity,[5] 331_2022_GBR-Cell-37C-B.raw.PG.Quantity,[6] 331_2022_GBR-Cell-37C-C.raw.PG.Quantity,[7] 331_2022_GBR-Cell-40C-A.raw.PG.Quantity,[8] 331_2022_GBR-Cell-40C-B.raw.PG.Quantity,[9] 331_2022_GBR-Cell-40C-C.raw.PG.Quantity,[10] 331_2022_GBR-SN-34C-A.raw.PG.Quantity,[11] 331_2022_GBR-SN-34C-B.raw.PG.Quantity,[12] 331_2022_GBR-SN-34C-C.raw.PG.Quantity,[13] 331_2022_GBR-SN-37C-A.raw.PG.Quantity,[14] 331_2022_GBR-SN-37C-B.raw.PG.Quantity,[15] 331_2022_GBR-SN-37C-C.raw.PG.Quantity,[16] 331_2022_GBR-SN-40C-A.raw.PG.Quantity,[17] 331_2022_GBR-SN-40C-B.raw.PG.Quantity,[18] 331_2022_GBR-SN-40C-C.raw.PG.Quantity
0,0,Tb427_000008800.1-p1,551307.562500,704164.562500,748118.875000,681518.625000,703385.687500,686961.1250,816417.812500,631893.750000,686818.437500,2.036847e+05,189815.296875,83305.46875,2.046948e+05,167546.3125,117179.421875,215901.4375,273887.625,135870.109375
1,29,Tb427_000198100.1-p1,17201.074219,22934.291016,24707.408203,20580.830078,23038.123047,24520.1875,29426.867188,28961.535156,100298.835938,1.619666e+06,787221.187500,106877.75000,1.162788e+06,311610.7500,151712.437500,565188.1875,584564.250,379278.843750


In [ ]:
#| hide
unexpected=0
pai_res = []
for pg in df_prot['PG.ProteinGroups']:
    prot = pg.split(';')[0]
    seq = fasta_dict[prot]
    seq = seq.replace('*','')
    observable_peptide = get_observable(seq)
    observed_peptide = set(df_pep[df_pep['PG.ProteinAccessions']==pg]['PEP.StrippedSequence'].unique())  
    print('observable_peptide',len(observable_peptide))
    print(observable_peptide)
    print('observed_peptide',len(observed_peptide))
    print(observed_peptide)
    pai = len(observed_peptide)/float(len(observable_peptide))
    pai_res.append(pai)
    print('pai',pai)
    if len(observed_peptide-observable_peptide) > 0:
        print (prot)
        print (fasta_dict[prot])
        print('observed_peptide-observable_peptide')
        print(observed_peptide-observable_peptide)
        print('observable_peptide')
        print(observable_peptide)
        print('observed_peptide')
        print(observed_peptide)
        unexpected+=1
        print('---------------------------\n\n') 
    print('---------------------------\n\n') 
print('unexpected',unexpected)

observable_peptide 13
{'NDWIG', 'EHVQR', 'GSEEWVR', 'SPPTGK', 'SCWTEICFEER', 'CYWFFNVGYR', 'AEYLQTVVNMECNSGR', 'PYLSGGGLIAR', 'LPHLLK', 'FEYAFGELLK', 'MTTSECYAPQLDLLMNSDHK', 'VWAFK', 'TTSECYAPQLDLLMNSDHK'}
observed_peptide 2
{'MTTSECYAPQLDLLMNSDHK', 'TTSECYAPQLDLLMNSDHK'}
pai 0.15384615384615385
---------------------------


observable_peptide 69
{'SISQSSR', 'IGFSEGFVSSR', 'ELEIVR', 'YAELENMHNIFVEK', 'MLEAEAK', 'LDEEFR', 'FELVQIQQELQTTR', 'QLAEVEGR', 'QESEILIR', 'MSISESDSLWLQK', 'IISSK', 'STQAK', 'NALQAAER', 'SNSPVPTNLTHLVTNGQLTVK', 'EQITK', 'SGSFSEEITNLQHR', 'FINSTSQR', 'AAEELSAAAALIVQR', 'QLEQLR', 'SEALDSLMATNNDR', 'EEACR', 'ASAADK', 'QMTQEHEK', 'SISESDSLWLQK', 'LPIAR', 'SVEQR', 'IAQLK', 'VFTISGFDGTELLEK', 'LEDAECDR', 'EQLDEAEAK', 'IGVVAEEVK', 'GCWILPEAYVHESTK', 'AENMEIELAAVTSR', 'WSGNQR', 'TQHLQQQYDLTETVYSQDMLELK', 'SELLQAEER', 'LVELIYPVK', 'GIQSANVPPSASK', 'AVTWDK', 'LQHAVEQASQTDK', 'CEELQR', 'EEHCR', 'EEQLSHLESEK', 'CSLLEK', 'QSVSNASFASLHEIDELR', 'LPYATVAECK', 'SIHLADPIPEATGEGVETD

In [ ]:
#| export
def add_pai(
    prot_df_path: Union[str, Path],
    pep_df_path: Union[str, Path],
    fasta_path: Union[str, Path],
    output_path: Union[str, Path] = None,
    pg_in_prot: str = 'PG.ProteinGroups',
    pg_in_pep: str = 'PG.ProteinAccessions',
    missed_cleavages_col: str = 'PEP.NrOfMissedCleavages',
    pep_stripped_col: str = 'PEP.StrippedSequence',
    enzyme: str = 'trypsin_full',
    filter_missed_cleavages: bool = True,
) -> pd.DataFrame:
    """
    Calculate Protein Abundance Index (PAI) for each protein group.
    
    Computes the ratio of observed peptides to observable peptides for
    each protein, providing a measure of sequence coverage that normalizes
    for protein size. This metric is useful for distinguishing between
    peptide-level and protein-level evidence in proteomics experiments.
    
    Parameters
    ----------
    prot_df_path : str or Path
        Path to protein-level report file (TSV format).
    pep_df_path : str or Path
        Path to peptide-level report file (TSV format).
    fasta_path : str or Path
        Path to FASTA file containing protein sequences.
    output_path : str or Path or None
        Path for output file. If None, appends '.pai' before the
        extension of prot_df_path. Default is None.
    pg_in_prot : str
        Column name for protein groups in protein report.
        Default is PG.ProteinGroups.
    pg_in_pep : str
        Column name for protein accessions in peptide report.
        Default is PG.ProteinAccessions.
    missed_cleavages_col : str
        Column name for missed cleavages count in peptide report.
        Default is PEP.NrOfMissedCleavages.
    pep_stripped_col : str
        Column name for stripped peptide sequences.
        Default is PEP.StrippedSequence.
    enzyme : str
        Enzyme name for in silico digestion to calculate observable
        peptides. Default is trypsin_full.
    filter_missed_cleavages : bool
        If True, only consider peptides with zero missed cleavages
        for observed count. Default is True.
    
    Returns
    -------
    pd.DataFrame
        Protein report DataFrame with added pai column containing
        the ratio of observed to observable peptides.
    
    Notes
    -----
    The PAI (Protein Abundance Index) is calculated as:
    
        PAI = N_observed / N_observable
    
    Where:
    
    - N_observed: Unique peptides detected for the protein
    - N_observable: Theoretical peptides from in silico digestion
    
    This differs from emPAI which uses the exponential formula.
    PAI values range from 0 to 1, where higher values indicate
    better sequence coverage.
    
    The output file is saved as TSV with the same structure as
    the input protein report plus the pai column.
    
    """
    # Convert paths
    prot_df_path = Path(prot_df_path)
    pep_df_path = Path(pep_df_path)
    fasta_path = Path(fasta_path)
    
    # Validate input files exist
    for path, name in [(prot_df_path, "Protein report"), 
                       (pep_df_path, "Peptide report"), 
                       (fasta_path, "FASTA")]:
        if not path.exists():
            raise FileNotFoundError(f"{name} file not found: {path}")
    
    # Set default output path
    if output_path is None:
        output_path = prot_df_path.with_suffix('.pai.tsv')
    else:
        output_path = Path(output_path)
    
    # Ensure output directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Load data
    df_prot = pl.read_csv(prot_df_path, separator='\t').to_pandas()
    df_pep = pl.read_csv(pep_df_path, separator='\t').to_pandas()
    
    # Validate required columns
    for col, df, name in [
        (pg_in_prot, df_prot, "protein report"),
        (pg_in_pep, df_pep, "peptide report"),
        (pep_stripped_col, df_pep, "peptide report"),
    ]:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in {name}")
    
    if filter_missed_cleavages:
        if missed_cleavages_col not in df_pep.columns:
            raise ValueError(
                f"Column '{missed_cleavages_col}' not found in peptide report. "
                f"Set filter_missed_cleavages=False to skip this filter."
            )
        df_pep = df_pep[df_pep[missed_cleavages_col] == 0]
    
    # Load FASTA sequences
    fasta_dict = load_fasta(fasta_path)
    
    # Calculate PAI for each protein group
    pai_results = []
    
    for protein_group in df_prot[pg_in_prot]:
        # Get first protein in group
        protein_id = protein_group.split(';')[0]
        
        # Get sequence and clean it
        if protein_id not in fasta_dict:
            pai_results.append(float('nan'))
            continue
            
        sequence = fasta_dict[protein_id].replace('*', '')
        
        # Calculate observable peptides
        observable_peptides = get_observable(sequence, enzyme=enzyme)
        n_observable = len(observable_peptides)
        
        # Get observed peptides for this protein group
        mask = df_pep[pg_in_pep] == protein_group
        observed_peptides = set(df_pep.loc[mask, pep_stripped_col].unique())
        n_observed = len(observed_peptides)
        
        # Calculate PAI
        if n_observable > 0:
            pai = n_observed / n_observable
        else:
            pai = float('nan')
        
        pai_results.append(pai)
    
    # Add PAI column to DataFrame
    df_prot['pai'] = pai_results
    
    # Save results
    df_prot.to_csv(output_path, sep='\t', index=False)
    
    return df_prot

In [ ]:
add_pai(prot_df_path = TEST_DATA/'pipeline_test/prot_report.tsv',
    pep_df_path =  TEST_DATA/'pipeline_test/pep_report.tsv',
    fasta_path = TEST_DATA/'pipeline_test/prot.fa')  

/Users/mtinti/git_projects/protein_cutter/nbs


,,PG.ProteinGroups,[1] 331_2022_GBR-Cell-34C-A.raw.PG.Quantity,[2] 331_2022_GBR-Cell-34C-B.raw.PG.Quantity,[3] 331_2022_GBR-Cell-34C-C.raw.PG.Quantity,[4] 331_2022_GBR-Cell-37C-A.raw.PG.Quantity,[5] 331_2022_GBR-Cell-37C-B.raw.PG.Quantity,[6] 331_2022_GBR-Cell-37C-C.raw.PG.Quantity,[7] 331_2022_GBR-Cell-40C-A.raw.PG.Quantity,[8] 331_2022_GBR-Cell-40C-B.raw.PG.Quantity,...,[10] 331_2022_GBR-SN-34C-A.raw.PG.Quantity,[11] 331_2022_GBR-SN-34C-B.raw.PG.Quantity,[12] 331_2022_GBR-SN-34C-C.raw.PG.Quantity,[13] 331_2022_GBR-SN-37C-A.raw.PG.Quantity,[14] 331_2022_GBR-SN-37C-B.raw.PG.Quantity,[15] 331_2022_GBR-SN-37C-C.raw.PG.Quantity,[16] 331_2022_GBR-SN-40C-A.raw.PG.Quantity,[17] 331_2022_GBR-SN-40C-B.raw.PG.Quantity,[18] 331_2022_GBR-SN-40C-C.raw.PG.Quantity,pai
0,0,Tb427_000008800.1-p1,551307.562500,704164.562500,748118.875000,681518.625000,703385.687500,686961.1250,816417.812500,631893.750000,...,2.036847e+05,189815.296875,83305.46875,2.046948e+05,167546.3125,117179.421875,215901.4375,273887.625,135870.109375,0.153846
1,29,Tb427_000198100.1-p1,17201.074219,22934.291016,24707.408203,20580.830078,23038.123047,24520.1875,29426.867188,28961.535156,...,1.619666e+06,787221.187500,106877.75000,1.162788e+06,311610.7500,151712.437500,565188.1875,584564.250,379278.843750,0.318841


In [ ]:
# pipline
# flag_proprietary_from_pg protein to filter out uniprots
# flag_proprietary_from_pg peptide to filter out uniprots
# load filtter prot and pep dataset
# load fasta 
# add pai to protein group

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()